In [ ]:
import pandas as pd
import plotly.express as px
import glob
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata, NearestNDInterpolator

In [ ]:
df = pd.read_csv("./resources/fabian_examples/surfdef_tpv36.csv", comment='#', delim_whitespace=True)
# df['years'], df['days'], df['hours'], df['seconds'] = convert_seconds_to_time(df['t'])
df['x [km]'] = df.x/1000
df['z [km]'] = df.z/1000
df

In [ ]:
size_domain_x = df['x'].max() - df['x'].min()
size_domain_z = df['z'].max() - df['z'].min()
print(f"x_size={size_domain_x} ({df['x'].min()}, {df['x'].max()}), z_size={size_domain_z} ({df['z'].min()}, {df['z'].max()})")

In [ ]:
%%time
# interpolation
grid_resolution = 500  # Number of grid points along each axis

# Create a regular grid
x_grid = np.linspace(-30000.0, 30000.0, grid_resolution)
z_grid = np.linspace(-15000.0, 50000.0, grid_resolution)
x_mesh, z_mesh = np.meshgrid(x_grid, z_grid)

# Interpolate h-disp values onto the grid
v_disp_grid = griddata(
    (df['x'], df['z']),  # Input points
    df['v-disp'],        # Input values
    (x_mesh, z_mesh),    # Grid to interpolate onto
    method='linear'      # Interpolation method ('linear', 'nearest', 'cubic')
)

# Extrapolate missing values using nearest neighbor
valid_mask = ~np.isnan(v_disp_grid)  # Mask of valid (non-NaN) values
valid_points = np.column_stack((x_mesh[valid_mask], z_mesh[valid_mask]))  # Valid (x, z) points
valid_values = v_disp_grid[valid_mask]  # Valid v-disp values

# Create a nearest neighbor interpolator
nearest_interp = NearestNDInterpolator(valid_points, valid_values)

# Fill missing values by extrapolating
v_disp_grid_filled = np.where(
    np.isnan(v_disp_grid),  # Condition: Where values are NaN
    nearest_interp(x_mesh, z_mesh),  # Fill with nearest neighbor values
    v_disp_grid  # Keep original values where not NaN
)

# Flatten the grid arrays
x_flat = x_mesh.flatten()
z_flat = z_mesh.flatten()
v_disp_flat = v_disp_grid_filled.flatten()

# Create a new DataFrame
grid_df = pd.DataFrame({
    'x': x_flat,
    'z': z_flat,
    'v-disp': v_disp_flat
})


# Display the new DataFrame
grid_df

In [ ]:
plt.figure(figsize=(8, 8))  # Set the figure size to 800x800 pixels

# Create the scatter plot
scatter = plt.scatter(
    grid_df['x'],  # x-axis data
    grid_df['z'],  # y-axis data
    c=grid_df['v-disp'],  # Color by h-disp
    cmap='RdBu_r',  # Use the reversed RdBu colormap (blue for negative, red for positive)
    vmin=grid_df['v-disp'].min(),  # Set the minimum value for the colormap
    vmax=grid_df['v-disp'].max(),
    s=0.1# Set the maximum value for the colormap
)

# Add a colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('v-disp')  # Add a label to the colorbar

# Add titles and labels
plt.title('Scatter Plot of x vs z colored by v-disp, interp data')
plt.xlabel('x')
plt.ylabel('z')

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))  # Set the figure size to 800x800 pixels

# Create the scatter plot
scatter = plt.scatter(
    df['x'],  # x-axis data
    df['z'],  # y-axis data
    c=df['v-disp'],  # Color by h-disp
    cmap='RdBu_r',  # Use the reversed RdBu colormap (blue for negative, red for positive)
    vmin=df['v-disp'].min(),  # Set the minimum value for the colormap
    vmax=df['v-disp'].max(),
    s=0.1# Set the maximum value for the colormap
)

# Add a colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('v-disp')  # Add a label to the colorbar

# Add titles and labels
plt.title('Scatter Plot of x vs z colored by v-disp, original data')
plt.xlabel('x')
plt.ylabel('z')

# Show the plot
plt.show()

# Plotly

In [ ]:
fig = go.Figure(data=go.Heatmap(
    x=grid_df['x'],  # x-axis grid points
    y=grid_df['z'],  # z-axis grid points
    z=grid_df['v-disp'],  # Gridded v-disp values
    colorscale='RdBu_r',  # Reversed RdBu colormap
    colorbar=dict(title='v-disp')  # Add a colorbar with a title
))

# Update layout
fig.update_layout(
    title='Heatmap of x vs z colored by v-disp (Regridded Data)',
    xaxis=dict(title='x'),
    yaxis=dict(title='z'),
    width=800,
    height=800,
    template='plotly_white'
)

fig.write_html("heatmap_plot.html")

# Show the plot
fig.show()

In [ ]:
v_disp_2d = grid_df['v-disp'].values.reshape((grid_resolution, grid_resolution))

# Create the surface plot
fig = go.Figure(data=go.Surface(
    x=x_mesh,  # x-axis grid points
    y=z_mesh,  # z-axis grid points
    z=v_disp_2d,  # 2D array of v-disp values
    colorscale='RdBu_r'  # Reversed RdBu colormap
))

# Update layout
fig.update_layout(
    title='Surface Plot of x vs z colored by v-disp (Regridded Data)',
    scene=dict(
        xaxis=dict(title='x'),
        yaxis=dict(title='z'),
        zaxis=dict(title='v-disp')
    ),
    width=800,
    height=800,
    template='plotly_white'
)

fig.write_html("3d_surface_plot.html")
# Show the plot
fig

In [ ]:
contour_fig = go.Figure(data=go.Contour(
    x=grid_df['x'],  # x-axis grid points
    y=grid_df['z'],  # z-axis grid points
    z=grid_df['v-disp'],  # Gridded v-disp values
    colorscale='RdBu_r',  # Reversed RdBu colormap
    colorbar=dict(title='v-disp'),  # Add a colorbar with a title
    contours=dict(
        start=grid_df['v-disp'].min(),  # Start of the contour levels
        end=grid_df['v-disp'].max(),  # End of the contour levels
        # size=10,  # Spacing between levels
        # showlabels=True,  # Show labels on the contour lines
        # labelfont=dict(size=12, color='black')  # Customize label font
    ),
    ncontours=100  # Number of contour levels (overrides `size` if specified)
))

# Update layout
contour_fig.update_layout(
    title='Contour Plot of x vs z colored by v-disp (Custom Levels)',
    xaxis=dict(title='x'),
    yaxis=dict(title='z'),
    width=800,
    height=800,
    template='plotly_white'
)

contour_fig.write_html("contour_plot.html")
# Show the plot
contour_fig

In [ ]:
fig = go.Figure(data=go.Scattergl(
    x=grid_df['x'],  # x-axis data
    y=grid_df['z'],  # y-axis data
    mode='markers',  # Use markers for scatter plot
    marker=dict(
        color=grid_df['v-disp'],  # Color by h-disp
        colorscale='RdBu_r',  # Blue-to-red gradient
        colorbar=dict(title='v-disp'),  # Add a colorbar with a title
        showscale=True,  # Show the color scale
        cmin=grid_df['v-disp'].min(),  # Set the minimum value for the color scale
        cmax=grid_df['v-disp'].max(),   # Set the maximum value for the color scale
    )
))

# Update layout for better visualization
fig.update_layout(
    title='Scatter Plot of x vs z colored by h-disp',
    xaxis=dict(title='x'),
    yaxis=dict(title='z'),
    template='plotly_white',  # Use a clean template
    width=800,  # Set the width of the figure to 800px
    height=800  # Set the height of the figure to 800px
)

fig.write_html("scatter_plot.html")
# Show the plot
fig

In [ ]:
# fig = go.Figure(data=go.Scattergl(
#     x=df['x'],  # x-axis data
#     y=df['z'],  # y-axis data
#     mode='markers',  # Use markers for scatter plot
#     marker=dict(
#         color=df['v-disp'],  # Color by h-disp
#         colorscale='RdBu_r',  # Blue-to-red gradient
#         colorbar=dict(title='v-disp'),  # Add a colorbar with a title
#         showscale=True,  # Show the color scale
#         cmin=df['v-disp'].min(),  # Set the minimum value for the color scale
#         cmax=df['v-disp'].max(),   # Set the maximum value for the color scale
#     )
# ))

# # Update layout for better visualization
# fig.update_layout(
#     title='Scatter Plot of x vs z colored by h-disp',
#     xaxis=dict(title='x'),
#     yaxis=dict(title='z'),
#     template='plotly_white',  # Use a clean template
#     width=800,  # Set the width of the figure to 800px
#     height=800  # Set the height of the figure to 800px
# )

# # Show the plot
# fig

In [ ]:
# # Define the number of bins for the heatmap
# num_x_bins = 50  # Number of bins along the x-axis
# num_z_bins = 50  # Number of bins along the z-axis

# # Create bins for x and z
# x_bins = np.linspace(df['x'].min(), df['x'].max(), num_x_bins)
# z_bins = np.linspace(df['z'].min(), df['z'].max(), num_z_bins)

# # Aggregate h-disp values into bins
# heatmap_data, x_edges, z_edges = np.histogram2d(
#     df['x'], df['z'], bins=[x_bins, z_bins], weights=df['v-disp'], density=False
# )

# # Create the heatmap
# fig = go.Figure(data=go.Heatmap(
#     x=x_edges,  # x-axis bin edges
#     y=z_edges,  # z-axis bin edges
#     z=heatmap_data.T,  # Aggregated h-disp values
#     colorscale='RdBu_r',  # Reversed RdBu colormap
#     colorbar=dict(title='v-disp')  # Add a colorbar with a title
# ))

# # Update layout
# fig.update_layout(
#     title='Heatmap of x vs z colored by v-disp',
#     xaxis=dict(title='x'),
#     yaxis=dict(title='z'),
#     width=800,
#     height=800,
#     template='plotly_white'
# )

# # Show the plot
# fig

In [ ]:
z=0
df_z0 = df[df['z'] == 1150.0].sort_values('x')
surface_height = px.line(df_z0, x='x [km]', y='v-disp', title='v-disp (sea surface height?) z=1.15km', markers=True)
surface_height

In [ ]:
x=0
df_x0 = df[df['x'] == 49.615].sort_values('z')
surface_height = px.line(df_x0, x='z [km]', y='v-disp', title='v-disp (sea surface height?)', markers=True)
surface_height

In [ ]:
shear_stress_fig = px.line(df, x='t', y='h-disp', title='shear stress', markers=True)
shear_stress_fig

In [ ]:
slip_fig = px.line(df, x='t', y='h-vel')
slip_fig

In [ ]:
slip_fig = px.line(df, x='days', y='slip')
slip_fig

In [ ]:
slip_rate_fig = px.line(df, x='t', y='slip_rate')
slip_rate_fig

In [ ]:
state_fig = px.line(df, x='years', y='state')
state_fig

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Assuming df is the DataFrame containing your data
fig = make_subplots(rows=2, cols=2, shared_xaxes=True,
                    subplot_titles=['Slip', 'Slip Rate', 'Shear Stress', 'State'],
                   vertical_spacing=0.08, horizontal_spacing=0.1)

# Add traces for each variable
fig.add_trace(go.Scatter(x=df['t'], y=df['slip'], mode='lines', name='Slip'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['t'], y=df['slip_rate'], mode='lines', name='Slip Rate'), row=2, col=1)
fig.add_trace(go.Scatter(x=df['t'], y=df['shear_stress'], mode='lines', name='Shear Stress'), row=1, col=2)
fig.add_trace(go.Scatter(x=df['t'], y=df['state'], mode='lines', name='State'), row=2, col=2)

# Update layout
fig.update_layout(title='Variables over Time', showlegend=False, height=1000)
fig.update_xaxes(matches='x')
# Show the figure
fig.show()